# Using Gemini's Function Calling

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
June 2025

-----

## Introduction

This notebook shows how to do [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) workflows with Large Language Models (LLMs) of Gemini. 

(The Raku package ["WWW::Gemini"](https://github.com/antononcube/Raku-WWW-Gemini) is used.)

-----

## Setup

Load packages:

In [15]:
use WWW::Gemini;
use JSON::Fast;

Choose a model:

In [16]:
my $model = "gemini-2.0-flash";
#my $model = "gemini-2.5-flash-preview-05-20";

gemini-2.0-flash

------

## Workflow

### Define a local function

Define a local function, i.e. _tool_:

In [17]:
sub get-current-weather(Str:D $location, Str:D $unit = "fahrenheit") returns Str {
    return "It is currently sunny in $location with a temperature of 72 degrees $unit.";
}

&get-current-weather

This is the tool description to be communicated to Gemini.

In [18]:
my %weather-function = %(
    name => 'get-current-weather',
    description => 'Get the current weather in a given location',
    parameters => %(
        type => 'object',
        properties => %(
            location => %(
                type => 'string',
                description => 'The city and state, e.g., Boston, MA'
            )
        ),
        required => ['location']
    )
);

{description => Get the current weather in a given location, name => get-current-weather, parameters => {properties => {location => {description => The city and state, e.g., Boston, MA, type => string}}, required => [location], type => object}}

### First communication with Gemini

In [19]:
# User prompt
my $prompt = 'What is the weather like in Boston, MA, USA?';

# Prepare the API request payload
my @messages = [{role => 'user',parts => [ %( text => $prompt ) ]}, ];

my @tools = [%weather-function, ];

[{description => Get the current weather in a given location, name => get-current-weather, parameters => {properties => {location => {description => The city and state, e.g., Boston, MA, type => string}}, required => [location], type => object}}]

Make the first Gemini API call:

In [ ]:
my $response = gemini-generate-content(
    @messages,
    :$model,
    :@tools
);

{candidates => [{avgLogprobs => -3.7914659414026473e-06, content => {parts => [{functionCall => {args => {location => Boston, MA}, name => get-current-weather}}], role => model}, finishReason => STOP, safetyRatings => [{category => HARM_CATEGORY_HATE_SPEECH, probability => NEGLIGIBLE} {category => HARM_CATEGORY_DANGEROUS_CONTENT, probability => NEGLIGIBLE} {category => HARM_CATEGORY_HARASSMENT, probability => NEGLIGIBLE} {category => HARM_CATEGORY_SEXUALLY_EXPLICIT, probability => NEGLIGIBLE}]}], modelVersion => gemini-2.0-flash, responseId => pVQ-aPvHEdHB7dcPwf7a4QY, usageMetadata => {candidatesTokenCount => 9, candidatesTokensDetails => [{modality => TEXT, tokenCount => 9}], promptTokenCount => 41, promptTokensDetails => [{modality => TEXT, tokenCount => 41}], totalTokenCount => 50}}

The response is already parsed from JSON to Raku. Here is its JSON form:

In [21]:
to-json($response)

{
  "modelVersion": "gemini-2.0-flash",
  "candidates": [
    {
      "avgLogprobs": -3.7914659414026473e-06,
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE"
        },
        {
          "probability": "NEGLIGIBLE",
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT"
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE"
        },
        {
          "probability": "NEGLIGIBLE",
          "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT"
        }
      ],
      "finishReason": "STOP",
      "content": {
        "role": "model",
        "parts": [
          {
            "functionCall": {
              "name": "get-current-weather",
              "args": {
                "location": "Boston, MA"
              }
            }
          }
        ]
      }
    }
  ],
  "usageMetadata": {
    "candidatesTokenCount": 9,
    "promptTokenCount": 41,
    "candi

### Refine the response with functional calls

The following copy of the messages is not required, but it makes repeated experiments easier:

In [42]:
my @messages2 = @messages;

[{parts => [text => What is the weather like in Boston, MA, USA?], role => user}]

Process the response -- invoke the tool, give the tool result to the LLM, get the LLM answer:

In [ ]:
my $assistant-message = $response<candidates>[0]<content>;
if $assistant-message<parts> {

    for |$assistant-message<parts> -> %part {
        if %part<functionCall> {
            
            @messages2.push($assistant-message);

            my $func-name = %part<functionCall><name>;
            my %args = %part<functionCall><args>;

            
            if $func-name eq 'get-current-weather' {
                my $location = %args<location>;
                my $weather = get-current-weather($location);

                my %function-response =
                            role => 'user',
                            parts => [{ 
                                functionResponse => {
                                    name => 'get-current-weather',
                                    response => %( content => $weather )
                                } 
                            }];

                @messages2.push(%function-response);
                
                # Send the second request with function result
                my $final-response = gemini-generate-content(
                    @messages2,
                    :@tools,
                    :$model,
                    format => "raku"
                );
                
                say "Assistant: ", $final-response<candidates>[0]<content><parts>».<text>.join("\n");
            }
        }
    }
} else {
    say "Assistant: $assistant-message<content>";
}

Assistant: :The weather in Boston, MA is currently sunny with a temperature of 72 degrees Fahrenheit.

